In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ShuffleSplit, RandomizedSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
test1 = pd.read_csv('../Data/Test1.csv', index_col=0)
test2 = pd.read_csv('../Data/Test2.csv', index_col=0)
test4 = pd.read_csv('../Data/Test4.csv', index_col=0)
data = pd.concat([test1, test2, test4])
data = data.sample(frac = 1) #shuffle 
data.head()

,a_enc_1,b_enc_1,c_enc_1,x_enc_1,y_enc_1,z_enc_1,a_enc_2,b_enc_2,c_enc_2,x_enc_2,y_enc_2,z_enc_2,fx_1,fy_1,fz_1,fx_2,fy_2,fz_2
t,,,,,,,,,,,,,,,,,,
1637005625,-90.000325,-0.000827,0.001000,233.953377,583.457877,-4.889781,89.996398,-0.000874,-179.997650,230.469347,577.247190,-3.941400,-6.723746,75.527686,992.072926,-310.824622,-399.468820,-855.050287
1637367637,-89.996232,-0.000854,0.000559,408.713052,1293.270912,-27.206371,89.994054,0.000291,-179.998000,408.583164,1298.457929,-20.645294,219.098536,-1221.110838,2401.327531,43.120577,832.862597,-563.837829
1637018922,-90.000617,-0.000722,0.001060,459.825426,794.310559,-100.166061,89.994277,-0.000322,-179.998048,459.742341,797.753569,-95.267698,321.074477,-1175.170662,2568.703542,31.489599,702.316667,-717.467852
1636592970,-89.998687,0.001300,0.004623,545.176525,281.828546,-95.917320,89.996577,-0.000907,179.998623,544.381629,285.386645,-91.226827,289.124851,-1126.036798,2546.102401,40.617402,680.771462,-752.157567
1636598086,-90.001400,-0.000434,0.000200,583.833987,227.406057,-146.759004,89.999061,-0.000320,179.999880,583.820940,230.706439,-141.567029,283.193578,-1262.497604,2608.084860,20.911292,734.955790,-770.167344


In [3]:
# train test split 
train = data.sample(frac = 0.8)
test = data.drop(train.index)

X_train = train.drop(['fx_1', 'fy_1', 'fz_1', 'fx_2', 'fy_2', 'fz_2'], axis=1)
y_train = train[['fx_1', 'fy_1', 'fz_1', 'fx_2', 'fy_2', 'fz_2']]
print(f"Training data shape X {X_train.shape} and y {y_train.shape}")

X_test = test.drop(['fx_1', 'fy_1', 'fz_1', 'fx_2', 'fy_2', 'fz_2'], axis=1)
y_test = test[['fx_1', 'fy_1', 'fz_1', 'fx_2', 'fy_2', 'fz_2']]
print(f"Testing data shape X {X_test.shape} and y {y_test.shape}")


Training data shape X (55391, 12) and y (55391, 6)
Testing data shape X (13848, 12) and y (13848, 6)


In [5]:
cv = ShuffleSplit(n_splits=5, test_size=.2, random_state=0)
model = LinearRegression()
cross_val_score(model, X_train, y_train, cv=cv)

array([0.59714627, 0.59062639, 0.60229568, 0.59751452, 0.59084583])

In [35]:
cv = ShuffleSplit(n_splits=5, test_size=.2, random_state=0)
model = KNeighborsRegressor()
cross_val_score(model, X_train, y_train, cv=cv)

array([0.94116684, 0.94065777, 0.93926709, 0.93909565, 0.94210869])

In [4]:
cv = ShuffleSplit(n_splits=5, test_size=.2, random_state=0)
model = RandomForestRegressor(max_depth=10, random_state=0)
cross_val_score(model, X_train, y_train, cv=cv)

array([0.9209645 , 0.92457165, 0.9227734 , 0.92162323, 0.92337623])

In [ ]:
model.fit(X_train, y_train)
pickle.dump(model, open('model.pkl', 'wb'))

In [14]:
print("Test set RMSE:", mean_squared_error(y_test, rf_random.predict(X_test), squared=False))

Test set RMSE: 69.67882053661698


/usr/local/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [8]:
random_grid = {'bootstrap': [True, False],
               'max_depth': [10, 20, 30],
               'min_samples_leaf': [1, 2, 4],
               'min_samples_split': [2, 5, 10],
               'n_estimators': [130, 180, 230]}

rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=0, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=4, min_samples_split=5, n_estimators=230; total time= 4.8min
[CV] END bootstrap=True, max_depth=30, min_samples_leaf=4, min_samples_split=2, n_estimators=230; total time= 3.2min
[CV] END bootstrap=False, max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=230; total time= 3.3min
[CV] END bootstrap=False, max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=130; total time= 2.0min
[CV] END bootstrap=False, max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=130; total time= 2.3min
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=2, min_samples_split=5, n_estimators=130; total time= 3.5min
[CV] END bootstrap=True, max_depth=20, min_samples_leaf=4, min_samples_split=10, n_estimators=130; total time= 1.7min
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=4, min_samples_split=5, n_estimators=180; total time= 3

RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [130, 180, 230]},
                   random_state=0, verbose=2)

In [7]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [9]:
filename = 'RandomForestRegressor.pkl'
rf_random = pickle.load(open(filename, 'rb'))

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test)

Model Performance
Average Error: 19.7045 degrees.
Accuracy = 96.54%.


In [23]:
predictions = best_random.predict(X_test)
# Mean squared error 
MSE = mean_squared_error(y_test, predictions)

# Root mean square error 
rms = np.sqrt(mean_squared_error(y_test, predictions))

# R squared
r2 = r2_score(y_test, predictions) 
r2

0.994163048495325